# <center>Estructura de Datos</center>
## <center>Universidad Nacional de Tres de Febrero</center>
## <center>TRZearcher</center>

### TRABAJO PRACTICO - TRANZAS DEL CODIGO
    Aguilera, Agustin
    Ambrosetti, Ramiro
    Noguera, Sol
    Tabarovsky, Javier

### TRZearcher
    Es un programa de busqueda y comparacion de precios de productos tecnologicos, para encontrar el mejor precio en el menor tiempo posible
    Por el momento trabajamos con: 
        - Compragamer
        - Full H4rd
        - Gezatek
        - Overdrive
        - Venex
    Estas paginas fueron seleccionadas debido a que son las mas economicas en su rubro a la fecha de publicacion de este informe, son paginas en continuo crecimiento, actualizadas diariamente, y con la mayor variedad de productos.

### UML - Diagrama de clases

<img src="UML.png">

### TECNICAS DE PROGRAMACION Y DISEÑO
    Las tecnicas de programacion utilizadas en este proyecto se basan en la programacion orientada a objetos, y la utilizacion de la libreria Scrapy.
    El diseño del programa esta dividido en 2 procesos, el Front y Back, esto se realizo de esta manera para solventar un problema.
    El problema se centraba en que al ejecutarse el programa, primero las spiders leian la informacion de configuracion y luego se ejecutaba la interfaz, esto ocasionaba que las 
    url de inicio se encontraban incompletas
    El diseño se basa en la modularizacion de las funcionalidades, a continuacion se puede ver un diagrama donde explicamos como se desenvuelve el programa.

<img src="Ilustracion.png">

# El programa comienza su ejecucion desde la clase Main, la cual llama a Interface y la ejecuta:

    1 - Una vez que el usuario eligio su configuracion de busqueda, y presiona el boton de buscar, estas configuraciones son guardadas en un temporal denominado temp_connector, y otro temporal con las url a las paginas completas denominado pages_complete.txt.
    2 - Este temp_conector es leido por la clase Proyect Connector.                           
    3 - La clase Proyect Connector, instancia a la clase Spider Connector la cual se encarga de :
        3.1 - Instanciar a la clase Orchestrator, la cual ejecuta las spiders seleccionadas, y estas mismas exportan los datos en temporales.csv.
        3.2 - Instanciar a la clase Sorter, la cual lee los temporales.csv, los ordena y filtra segun el tipo de busqueda, para luego instanciar a la clase Exporter con los resultados.
    4 - La clase Exporter se encarga de escribir los resultados en dos formatos, csv, json y html.
    Esta seria una explicacion a grandes rasgos del funcionamiento del programa       

### INTERFACE (FRONT)
    La interfaz esta creada como un proceso separado del programa, con una unica clase la cual crea una interfaz grafica con el uso de la libreria tkinter.
    En la cual a la hora de presionar el boton de buscar se realizan las siguientes acciones:
    
        1. Verifica que la entrada del producto no puede estar vacia, y al menos se debe seleccionar una pagina.

In [ ]:
    def __code_button(self):
            if (str(self.product.get()).strip() == "") and (self.compra_gamer.get() + self.full_hard.get() + self.gezatek.get() + self.venex.get() + self.overdrive.get() <= 0):
                messagebox.showerror(message="Recordar que:\n - La entrada del producto no puede estar vacia\n - Al menos hay que seleccionar una pagina", title="Problema con TRZearcher")
            else:
                self.__write_data_connector()
                self.__write_pages()
                subprocess.call("python ./Back/Connector/proyect_connector.py", shell=True)
                self.window.destroy()

        2. Escribe los datos de la interfaz en un temporal, llamado temp_connector.txt, el cual luego es leido por el otro proceso.

In [ ]:
    def __write_data_connector(self):
        with open("./Back/Connector/temp_connector.txt", 'w') as pages_file:
            pages_file.write(str(self.product.get()) + "\n")
            pages_file.write(str(self.compra_gamer.get()) + "\n")
            pages_file.write(str(self.full_hard.get()) + "\n")
            pages_file.write(str(self.gezatek.get()) + "\n")
            pages_file.write(str(self.venex.get()) + "\n")
            pages_file.write(str(self.overdrive.get()) + "\n")
            pages_file.write(str(self.option.get()) + "\n")

        3. Completa las paginas a buscar, leyendo de pages_initials.txt, agregando el producto al final y lo exporta a un temporal llamado page_complete.txt.

In [ ]:
    def __write_pages(self):
        pages_complete = []
        number_of_pages = 0
        fileInitials = os.path.abspath("./Back/Searcher/Data/pages_initials.txt")
        fileComplete = os.path.abspath("./Back/Searcher/Data/pages_complete.txt")
        with open(fileInitials, "r") as pages_file:
            for page in pages_file:
                number_of_pages += 1
                page = page.rstrip('\n') + str(self.product.get()) + "\n"
                pages_complete.append(page)
        with open(fileComplete, "w") as pages_file:
            for x in range(0, number_of_pages):
                pages_file.write(pages_complete[x])

        Una vez finalizado, se ejecuta el siguiente proceso, y se cierra la ventana.

### CONNECTOR (BACK)
    El modulo connector esta creado para la conexion del programa, la cual esta compuesta de dos clases proyect_connector y spyder_connector.
        1. proyect_connector: Se basa en la lectura de los datos obtenidos de interfaz (a travez del temporal temp_connector.txt).

In [ ]:
    __data = []
    with open("./Back/Connector/temp_connector.txt", "r") as data_connector:
        for data in data_connector:
            __data.append(str(data).strip())

    Connector(__data[0], __data[1], __data[2], __data[3], __data[4], __data[5], __data[6]).execute_connector()

        2. spyder_connector: Es la clase mas importante del programa, la cual es la encargada en primer instancia de verificar que paginas se quieren buscar, para proceder a ejecutar las Spiders, a su vez elimina los archivos temporales.

In [ ]:
    class Connector:

        def __init__(self, product, compragamer, fullhard, gezatek, venex, overdrive, type_search):
            self.product = product
            self.pages = self.__spiders_validation(compragamer, fullhard, gezatek, venex, overdrive)
            self.type_search = type_search

        def __spiders_validation(self, compragamer, fullhard, gezatek, venex, overdrive):
            pages = []
            if int(compragamer) == 1:
                pages.append("compragamer")
            if int(fullhard) == 1:
                pages.append("fullh4rd")
            if int(gezatek) == 1:
                pages.append("gezatek")
            if int(overdrive) == 1:
                pages.append("overdrive")
            if int(venex) == 1:
                pages.append("venex")
            return pages

        def execute_connector(self):
            Orchestrator().execute_spiders(self.pages)
            Sorter(self.pages, self.product, self.type_search).execute_sorter()
            self.__clean()
            sys.exit()

        def __clean(self):
            os.remove("./Back/Connector/temp_connector.txt")
            os.remove("./Back/Searcher/Data/pages_complete.txt")
            for page in self.pages:
                os.remove(page + ".csv")

### PROCESSOR (BACK)
    Completar por Sol

In [ ]:
Codigo importante a explicar

### SEARCHER (BACK)
    El modulo searcher es el encargado de recuperar los datos de la web, se divide a su vez en:
        1. Config: Configuracion de scrapy, con sus items, etc.
        2. Data: Archivos temporales y estaticos, los cuales son utilizados por las arañas para su funcionamiento.
        3. Spiders: Son las encargadas de recuperar la informacion de determinadas paginas, y exportarlas a csv temporales. Este modulo incluye su orchestrator, el cual las ejecuta.
        Un ejemplo podria ser la CompragamerSpider, las demas son similares.

    La primer parte de cada spider, se basa en su configuracion:
        1. name: Nombre de la araña.
        2. custom_settings: Forma de exportacion de los datos, en este caso csv.
        3. allowed_domains: Referencia al dominio en el cual se va a trabajar, del cual no se puede salir.
        4. item_count: Es una variable que utilizaremos mas adelante para limitar la salida de datos, y que el programa no tarde demasiado tiempo.
        5. start_urls: Links de las paginas desde donde se va a lanzar a recolectar datos, en este caso se toman desde un archivo temporal.
        6. rules: Reglas que la spider debe tener en cuenta en este caso, es que cada vez que encuentre un elemento de la lista (xpaths), entre a el dominio ejecutando parse_items,
            esto es muy util ya que al entrar a cada publicacion se puede extraer informacion que no se encuentra en la lista de la url incial. Otra regla podria ser que cada vez que encuentre
            un xpaths de un boton (por ejemplo siguiente pagina), siga esa url, para continuar con la busqueda.
        Por ultimo elimina el archivo temporal anterior, en el caso de que todavia siga existiendo (esto se hace si por alguna razon el programa se interrumpe en su proceso).

In [ ]:
    class CompragamerSpider(CrawlSpider):
        """
            Spider que recolecta datos de la pagina www.compragamer.com, con un limite de 15
        """

        name = 'compragamer'

        custom_settings = {'FEEDS': {'compragamer.csv': {'format': 'csv'}}}

        allowed_domains = ['compragamer.com']

        item_count = 0

        url = ""
        filePath = os.path.abspath("../TRZearcher/Back/Searcher/Data/pages_complete.txt")
        with open(filePath, "r") as pages:
            for page in pages:
                if page.find("compragamer") > 0:
                    url = page

        start_urls = [url]

        rules = {
            Rule(LinkExtractor(allow=(), restrict_xpaths=('//*[@class="products__item"]')),
                 callback='parse_item', follow=False)
        }

        try:
            os.remove('compragamer.csv')
        except OSError:
            pass

    La segunda parte de cada spider, se basa en la recuperacion de datos:
        - La llamada al TrzpidersItem se da para guardar los elementos que vamos a recuperar.
        - Cada elemento se recupera en mediante css (menos link y time), y dependiendo como se encuentre se arregla para su posterior salida en csv.
        - Los elementos a extraer por pagina se limitan a un maximo de 15 elementos, para que el programa no tarde demasiado tiempo y recupere muchos elementos.

In [ ]:
    def parse_item(self, response):
                """
                    Recolecta la informacion de cada item
                """

                item = TrzpidersItem()

                item['title'] = str(response.css(
                    '.filaNombre div::text').extract_first()).strip()

                item['price'] = str(response.css(
                    '.col-xs-5::text').extract_first()).split("$")[1].strip()

                item['category'] = str(response.css(
                    '.detalleNombre .product-card__name::text').extract_first()).capitalize().strip()

                item['link'] = str(response.url)

                item['time'] = (str(datetime.now().year) + "-" + str(datetime.now().month) + "-" + str(
                    datetime.now().day) + " " + str(datetime.now().hour) + ":" + str(datetime.now().minute))

                self.item_count += 1
                if self.item_count > 15:
                    raise CloseSpider('item exceeded')

                yield item

### TEST (BACK)
    Completar por Javi

In [ ]:
Codigo importante a explicar